In [10]:
%pip install imblearn

/bin/bash: /home/sukhmeet/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


# Import Section

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.models import load_model
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score



# Loading Dataset

In [12]:
FILE_PATH_TRAINING = "../dataset/UNSW_NB15_training-set.csv"
FILE_PATH_TESTING = "../dataset/UNSW_NB15_testing-set.csv"

df_training =   pd.read_csv(FILE_PATH_TRAINING, index_col=0)
df_testing =    pd.read_csv(FILE_PATH_TESTING, index_col=0)

In [13]:
df_training.shape, df_testing.shape

((175341, 44), (82332, 44))

In [22]:
# create a boolean mask of missing values
missing_values_mask = df_training.isnull()

# count the number of missing values in each column
missing_values_count = missing_values_mask.sum()

# print the result
print(missing_values_count)


dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat           0
label      

In [31]:
# value counts
unique_values = {}
for col in df_training.columns:
    if df_training[col].dtype == object:
        print(df_training[col].value_counts())
        print()
        unique_values[col] = df_training[col].unique()

unique_values_test = {}
for col in df_testing.columns:
    if df_testing[col].dtype == object:
        print(df_testing[col].value_counts())
        print()
        unique_values_test[col] = df_testing[col].unique()

# swap
# unique_values, unique_values_test = unique_values_test, unique_values
# vlaues which are in test but not in train
for col in unique_values_test.keys():
    for val in unique_values_test[col]:
        if val not in unique_values[col]:
            print(col, val)

proto
tcp       79946
udp       63283
unas      12084
arp        2859
ospf       2595
          ...  
rdp          98
netblt       98
igmp         18
icmp         15
rtp           1
Name: count, Length: 133, dtype: int64

service
-           94168
dns         47294
http        18724
smtp         5058
ftp-data     3995
ftp          3428
ssh          1302
pop3         1105
dhcp           94
snmp           80
ssl            56
irc            25
radius         12
Name: count, dtype: int64

state
INT    82275
FIN    77825
CON    13152
REQ     1991
RST       83
ECO       12
PAR        1
URN        1
no         1
Name: count, dtype: int64

attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64

proto
tcp           43095
udp           29418
unas           3515
arp            

In [ ]:
df_training.head()

# print the columns
for idx, col in enumerate(df_training.columns):
    print(idx, col)

In [ ]:
df_training.head()

In [ ]:
print(df_training.columns, sep="\n")

In [ ]:
for col in df_training.columns:
    if 50 >=df_training[col].nunique() :
        print(col, df_training[col].nunique())

# EDA

In [ ]:
df_training.info()

In [ ]:
df_training.describe()
# before log transformation

In [ ]:
df_training.describe()
# After log transformation

In [ ]:
def plot_normal_distribution(df):
    for column in df.columns:
        sns.displot(df[column])
        plt.show()
        
plot_normal_distribution(df_training)

# Data Preprocessing

## Removing columns

In [ ]:
columns_to_remove = ['proto','state','service','attack_cat']
df_training.drop(columns_to_remove, axis=1, inplace=True)
df_testing.drop(columns_to_remove, axis=1,inplace=True)


### Remove columns for multi-class classification

In [ ]:
columns_to_remove = ['proto','state','service','label']
df_training.drop(columns_to_remove, axis=1, inplace=True)
df_testing.drop(columns_to_remove, axis=1,inplace=True)

## Log Modification

In [ ]:
for column in df_training.columns:
    if column !='label' and df_training[column].nunique() > 50:
        df_training[column] = np.log(df_training[column]+1)
        df_testing[column] = np.log(df_testing[column]+1)
    

## Data Standardization

In [ ]:
standard_scaler = StandardScaler()

for column in df_training.columns:
    if column != 'label' and column != 'attack_cat':
        standard_scaler.fit(df_training[column].values.reshape(-1,1))
        df_training[column] = standard_scaler.transform(df_training[column].values.reshape(-1,1))
        df_testing[column] = standard_scaler.transform(df_testing[column].values.reshape(-1,1))

In [ ]:
# after standardization
df_training.describe()

# Train Test Split

## Splitting

In [ ]:
X_train = df_training.drop('label', axis=1)
y_train = df_training['label']

X_test = df_testing.drop('label', axis=1)
y_test = df_testing['label']



### Spliting for multiclass classification

In [ ]:
X_train = df_training.drop('attack_cat', axis=1)
y_train = df_training['attack_cat']

X_test = df_testing.drop('attack_cat', axis=1)
y_test = df_testing['attack_cat']

### Encoding for multiclass classification

In [ ]:
X_train.head()

In [ ]:
y_train[:50]

In [ ]:
# encode attack_cat using label encoder
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)
y_test = labelencoder_y.transform(y_test)


# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)


In [ ]:
f={}
for i in range(len(y_train)):
    if y_train[i] not in f:
        f[y_train[i]]=1
    else:
        f[y_train[i]]+=1
print(f)

In [ ]:
# Calculate class weights to handle class imbalance
class_counts = np.bincount(y_train)
class_weights = dict(enumerate((max(class_counts) / class_counts).astype(float)))
print(class_weights)


### Random Oversampling for multiclass classification

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Create a RandomOverSampler object
oversampler = RandomOverSampler()

# Fit and transform the data
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)


In [ ]:
X_train, y_train = X_train_oversampled, y_train_oversampled

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

## Validation split

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [ ]:
X_train.shape,y_train.shape,X_val.shape,y_val.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Fit a Random Forest model using class weights
rf = RandomForestClassifier(n_estimators=100, n_jobs=50, random_state=42, verbose=1, criterion='entropy',warm_start=False)
rf.fit(X_train, y_train)



In [ ]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
print(classification_report(y_test, y_pred))
# plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');


In [ ]:

# Get feature importances
feature_importances = rf.feature_importances_

In [ ]:
feature_importances

In [ ]:
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances}).sort_values('importance', ascending = False)


In [ ]:
# plot feature importances
plt.figure(figsize = (8, 6))
sns.barplot(x = 'importance', y = 'feature', data = feature_importances)
plt.title('Feature Importances')
plt.show()


In [ ]:
# select features with importance greater than 0.01
selected_features = feature_importances[feature_importances['importance'] > 0.05]['feature'].values


In [ ]:
# total number of features selected vs total number of features
len(selected_features), X_train.shape[1]

In [ ]:
# new training and testing data with selected features
X_train = X_train[selected_features]
X_test = X_test[selected_features]
X_val = X_val[selected_features]

In [ ]:
X_train.shape,X_test.shape,X_val.shape,y_train.shape,y_test.shape,y_val.shape

# Model ANN

## Model

In [ ]:
def get_model_ann():
  model = Sequential()
  model.add(Dense(50, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(25, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def get_model_ann_multiclass():
  model = Sequential()
  model.add(Dense(50, input_dim=X_train.shape[1], activation='relu'))
  model.add(Dense(25, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# model_ann = get_model_ann()  
model_ann = get_model_ann_multiclass()

## Train

In [ ]:
history_ann = model_ann.fit(X_train, y_train, epochs=50, batch_size=256, verbose=1, validation_data=(X_val, y_val))


## Predict

In [ ]:
# load model
model_ann = load_model('models/saved_models/model_ann_20231103-112820_0.887.keras')

### Binary prediction

In [ ]:
y_prediction_ann = model_ann.predict(X_test)


In [ ]:
threshold = 0.5
y_prediction_ann = np.where(y_prediction_ann > threshold, 1, 0)

In [ ]:
accuracyScore_ann = accuracy_score(y_test, y_prediction_ann)
print("ANN Accuracy: ", accuracyScore_ann)

In [ ]:
loss, accuracy = model_ann.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))
print('Loss: %f' % (loss*100))

### Multi ClassPredicition


In [ ]:
y_prediction_ann_multiclass = np.argmax(y_prediction_ann, axis=1)

In [ ]:
# calculate accuracy
accuracyScore_ann = accuracy_score(y_test, y_prediction_ann_multiclass)
print("ANN Accuracy with multiclass: ", accuracyScore_ann)

### Plot graph of probability of each class

In [ ]:
def plot_probabilities(y_prediction_array, true_label,predicted_label=None):
  if predicted_label is None:
    predicted_label = np.argmax(y_prediction_array)
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), y_prediction_array, color="#777777")
  plt.ylim([0, 1])
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

  plt.show()


for i in range(56, 60):
  plot_probabilities(y_prediction_ann[i], y_test[i], y_prediction_ann_multiclass[i])

In [ ]:
def plot_confusion_matrix(y_test, y_prediction, title):
    cm = confusion_matrix(y_test, y_prediction)
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(title)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()
plot_confusion_matrix(y_test, y_prediction_ann_multiclass, "ANN Confusion Matrix")

### Predict for multi-class classification

In [ ]:
# save model with time and accuracy
import datetime
model_ann.save('model_ann_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+str(accuracyScore_ann)[0:5]+'.h5')

## Confusion Matrix

In [ ]:
print(classification_report(y_test, y_prediction_ann))

In [ ]:
## give confusion matrix
cm_ann = confusion_matrix(y_test, y_prediction_ann)
##  plot it
plt.figure(figsize=(10,10))
plt.title('ANN Confusion Matrix')
sns.heatmap(cm_ann, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


## Roc Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_ann)
roc_auc = auc(fpr, tpr)



In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_pred)
print("AUC Score:", auc_score)


# Random Forest

In [ ]:

rf = RandomForestClassifier(n_estimators=100, n_jobs=50, random_state=42, verbose=1, criterion='entropy',warm_start=True)

rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# XGBoost

In [ ]:
# write code for xgboost
xgb = XGBClassifier(n_estimators=100, n_jobs=50, random_state=42, verbose=1, criterion='entropy',warm_start=True)
xgb.fit(X_train, y_train)


In [ ]:
#predict
y_pred = xgb.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# LSTM

In [ ]:
def get_model_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(25, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(10))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

model_lstm = get_model_lstm()


In [ ]:
# reshape data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)

In [ ]:
# fit model
history_lstm = model_lstm.fit(X_train, y_train, epochs=100, batch_size=1024, verbose=1, validation_data=(X_val, y_val))

In [ ]:
# evaluate model on test data
y_prediction_lstm = model_lstm.predict(X_test)

threshold = 0.5

y_prediction_lstm = np.where(y_prediction_lstm > threshold, 1, 0)

accuracyScore_lstm = accuracy_score(y_test, y_prediction_lstm)

print("LSTM Accuracy: ", accuracyScore_lstm)



In [ ]:
# save model with time and accuracy
import datetime
model_lstm.save('model_lstm_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+str(accuracyScore_lstm)[0:5]+'.h5')

In [ ]:
# evaluate model on test data
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy_lstm*100))
print('Loss: %f' % (loss_lstm*100))

In [ ]:
# plot loss during training
plt.figure(figsize=(10, 10))
plt.title('Loss')
plt.plot(history_lstm.history['loss'], label='train')
plt.plot(history_lstm.history['val_loss'], label='test')
plt.legend()
plt.show()


In [ ]:
# confusion matrix
def plot_confusion_matrix(y_test, y_prediction, title):
    cm = confusion_matrix(y_test, y_prediction)
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(title)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()

plot_confusion_matrix(y_test, y_prediction_lstm, "LSTM Confusion Matrix")
